<a href="https://colab.research.google.com/github/sakurakun999/AutoGenBook/blob/main/AutoGenBookJP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 仕様の策定

以下を入力した後，【すべてのセルを実行（Ctrl + F9）】を押してください．  

In [1]:
# @markdown ## 必須項目
# @markdown ### 教科書の内容
book_content = "マクロ経済とミクロ経済の基礎から応用まで幅広く取り扱った本" #@param {type:"string"}
# @markdown ### 大まかなページ数
n_pages = 40 # @param {"type":"integer","placeholder":"40"}
# @markdown ### 出力形式
tex_output = False #@param {type:"boolean"}
pdf_output = True #@param {type:"boolean"}
md_output = False #@param {type:"boolean"}

# @markdown ## 任意項目
# @markdown ### 想定する読者層
target_readers = "大学で経済学を受ける前に読む本" #@param {type:"string"}
# @markdown ### 数式の出現度合い
equation_frequency_level = 1 #@param {type:"slider", min:1, max:5, step:1}
# @markdown ### 内容出力時に章立て・構成の考慮
do_consider_outline = True #@param {type:"boolean"}
# @markdown ### 内容出力時に直前の出力を考慮
do_consider_previous_sections = True #@param {type:"boolean"}
# @markdown ### その他、内容についての要望
additional_requirements = "数式メインではなく、実際の社会での例を大切にして" #@param {type:"string"}

if book_content == "":
    print('\033[31m'+'教科書の内容を指定してください。'+'\033[0m')
if n_pages == 0:
    print('\033[31m'+'ページ数を指定してください．'+'\033[0m')

## プロンプトの定義

In [2]:
# 共通のプロンプト
prompt_common = f"""
以下の内容で本を執筆します．
{book_content}
本全体のページ数は{n_pages}ページ，1ページあたり40行を想定しています．ですます調で記述してください．
"""
if target_readers != "":
    prompt_common += f"想定読者としては以下を考えています．\n {target_readers}"
if additional_requirements != "":
    prompt_common += f"また，以下を考慮に入れてください．\n {additional_requirements}"

# 本・章のタイトルと概要の生成用プロンプト
prompt_book_title = prompt_common + """
以上をもとに，以下のようなjson形式で，本・章のタイトル，本・章の概要を示してください．
本の概要には，内容の要約だけではなく，本の主な目的やカバーする内容の範囲と深さなどについても触れてください．5から10文ほどで，詳細にお願いします．
また，各章に割くべきページ数を考えてください．ページ数は0.1単位で，0.8ページのように書いてください．
それに加え，内容の意味的凝集性から考えて，各章を分節化する必要がありますかどうか（needsSubdivision）を考えてください．trueかfalseで答えてください．
推測や未確認の情報は含めないでください．また，タイトルに第何章であるかは書かないでください．
節の数は必要に応じて変えてください．
```json
{{"title": "",
"summary": "",
"childs":
    [{{"title": "",
    "summary": "",
    "n_pages": ,
    "needsSubdivision":
    }},
    {{"title": "",
    "summary": "",
    "n_pages": ,
    "needsSubdivision":
    }},
    {{"title": "",
    "summary": "",
    "n_pages": ,
    "needsSubdivision":
    }}]
}}
```
"""

# 分節化用プロンプト
prompt_section_list_creation = prompt_common + """
以上の情報から，{book_title}というタイトルで本を作成しようと思います．本の概要を以下に示します．
{book_summary}
その中の{target}についての部分を{n_pages}ページで作成したいです．1ページあたり40行を想定しています．
この部分の概要は，以下です．
{section_summary}
この部分を分節化して，複数のパートに分けて欲しいです．
各パートのタイトルと概要を以下のようなjson形式にて出力してください．また，各パートに割くべきページ数を考えてください．ページ数は0.1単位で，0.8ページのように書いてください．
それに加え，内容の意味的凝集性から考えて，各章を分節化する必要がありますかどうか（needsSubdivision）を考えてください．trueかfalseで答えてください．
タイトルに第何章・節であるかは書かないでください．
```json
[{{"title": "",
"summary": "",
"n_pages": ,
"needsSubdivision":
}},
{{"title": "",
"summary": "",
"n_pages": ,
"needsSubdivision":
}}]
```
"""

# 本文の内容の生成用プロンプト
prompt_content_creation = prompt_common + """
以上の情報から，{book_title}というタイトルで本を作成しようと思います．本の概要を以下に示します．
{book_summary}
その中の{target}についての部分を{n_pages}ページで作成したいです．1ページあたり40行を想定しています．
この部分の概要は，以下です．
{section_summary}
{toc_and_summary}
{previous_sections}
その部分の内容を{n_pages}分，つまり{n_pages} × 40行分をLaTeXで出力してください．プリアンブルで必要なライブラリはすべてインポートされています．
推測や未確認の情報は含めないでください．見出しは必要なく，本文のみを出力してください．
{equation_frequency}
数式にはequation環境もしくはalign環境を用いて，それ以外の環境は用いないでください．
数式をネストしないように，すなわち\\[ \\begin{{align}} \\end{{align}} \\]ではなく，\\begin{{align}} \\end{{align}}とするよう気をつけてください．
プログラムを出力する場合はlstlisting環境を使ってください．オプションにはプログラミング言語のみを設定してください．例えば，使用言語がPythonであれば\\begin{{lstlisting}}[language=Python]のようにしてください．
#を使う場合は必ず\#のようにエスケープしてください．
出力形式は以下のようにお願いします．
```tex
本文の内容
```
"""

## パラメータ設定

In [3]:
# パラメータ
max_depth = 5 # 1なら節のみ，2なら小節まで．．．
max_output_pages = 1.5 # LLMの最大出力ページ数

book_node_name = "book" # ルートノードの名前

openai_api_secret_key_name = 'openai_api' # openai_apiをシークレットキーとして登録した時の変数名
model_name = "gpt-4o" # モデルの名前

if do_consider_previous_sections:
    n_previous_sections = 1
else:
    n_previous_sections = 0

## ライブラリのインストール・インポート

In [4]:
!apt-get update
!apt-get install -y python3-dev graphviz libgraphviz-dev pkg-config
!apt-get install -y latexmk
!apt-get install -y texlive-lang-japanese
!apt-get install -y texlive-latex-extra
!apt-get install -y texlive-science
%pip install -qU langchain-openai
%pip install pygraphviz
%pip install pylatex

import os
import re
import json
import networkx as nx
from google.colab import userdata
from IPython.display import Markdown
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator
from typing import List, Optional

import matplotlib.pyplot as plt
from networkx.drawing.nx_agraph import graphviz_layout

from pylatex import Command, Document, Section, Subsection, Package
from pylatex.section import Chapter
from pylatex.utils import NoEscape

from google.colab import files

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [3,633 kB]
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,197 kB]
Get:13 http://archive.ubuntu.com/ubunt

## グラフの作成

In [5]:
book_graph = nx.DiGraph(book_content=book_content, target_readers=target_readers, equation_frequency_level=equation_frequency_level, additional_requirements=additional_requirements)

## タイトル・各章の作成

### 関数の定義

In [6]:
def extract_book_and_chapter_contents(markdown_text):
    """
    Markdown形式のテキストから最初に見つかったJSONデータを抽出し、Pythonの辞書型に変換して返す関数。

    Args:
        markdown_text (str): Markdown形式の文字列。この文字列の中にJSON形式のデータが含まれていると想定される。

    Returns:
        dict or None: 正しい形式のJSONが見つかれば辞書型で返し、見つからなかった場合やパースに失敗した場合はNoneを返す。
    """

    # MarkdownからJSONの開始点を見つける
    start_index = markdown_text.find('{')
    if start_index == -1:
        return None

    # 文字列全体をトラバースし、ネストされた括弧のバランスをチェックする
    brace_count = 0
    for i in range(start_index, len(markdown_text)):
        if markdown_text[i] == '{':
            brace_count += 1
        elif markdown_text[i] == '}':
            brace_count -= 1

        # ブレースのバランスが取れたらその時点で抜き出す
        if brace_count == 0:
            json_string = markdown_text[start_index:i+1]
            try:
              # JSON形式に変換
              json_data = json.loads(json_string)
              return json_data
            except json.JSONDecodeError as e:
                print(f"JSONのパースエラー: {e}")
                return None
            return

    # 閉じ括弧が見つからない場合
    return None

### LLMによる出力

In [8]:
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = userdata.get(openai_api_secret_key_name)

llm = ChatOpenAI(model="gpt-4o")

prompt = PromptTemplate.from_template(prompt_book_title)

chain = prompt | llm
result = chain.invoke(
    {
        "book_content": book_content,
        "target_readers": target_readers,
        "n_pages": n_pages,
        "additional_requirements": additional_requirements
    }
)

book_json = extract_book_and_chapter_contents(result.content)

NotFoundError: Error code: 404 - {'error': {'message': 'The model `gpt-4o` does not exist or you do not have access to it.', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}

### 結果を本グラフへ格納

In [ ]:
# 本について
book_graph.add_nodes_from([(book_node_name, {"title": book_json["title"], "summary": book_json["summary"], "n_pages": n_pages, "needsSubdivision": True})])

# 章（部）について
book_graph.add_nodes_from([(str(idx+1), child) for idx, child in enumerate(book_json["childs"])])
book_graph.add_edges_from([(book_node_name, str(idx+1)) for idx in range(len(book_json["childs"]))])

## タイトルと構成の確認

### タイトルと構成の表示

In [ ]:
book_node = book_graph.nodes[book_node_name]

content_md = ""
content_md += "\n ## タイトル：" + book_node["title"] + "（ページ数：" + str(book_node["n_pages"]) + "）"
content_md += "\n " + book_node["summary"]
for idx, child_node_name in enumerate(book_graph.successors(book_node_name)):
    child_node = book_graph.nodes[child_node_name]
    content_md += "\n ### 第" + str(idx+1) + "章：" + child_node["title"] + "（ページ数：" + str(child_node["n_pages"]) + "）"
    content_md += "\n" + child_node["summary"]

Markdown(content_md)

## 本グラフの作成

### 関数の定義

In [ ]:
def extract_section_list(markdown_text):

    pattern = r'```json\s*(.*?)\s*```'
    match = re.search(pattern, markdown_text, re.DOTALL)

    if match:
        json_string = match.group(1)
        data = json.loads(json_string)
        return data
    else:
        print("JSONデータが見つかりませんでした。")
        return None

def extract_section_content(markdown_text):

    pattern = r'```tex\s*(.*?)\s*```'
    match = re.search(pattern, markdown_text, re.DOTALL)

    if match:
        tex_string = match.group(1)
        return tex_string
    else:
        print("TeXデータが見つかりませんでした。")
        return None

def get_equation_frequency(equation_frequency_level):
    if equation_frequency_level == 1:
        return "数式はほぼ使用せず、すべての概念を平易な言葉で説明してください。数式が絶対に必要な場合のみ、最小限の使用に留めてください。"
    elif equation_frequency_level == 2:
        return "数式は控えめに使用し、主に文章で説明を行ってください。必要な場合のみ簡単な数式を用いてください。"
    elif equation_frequency_level == 3:
        return "数式と文章による説明をバランス良く組み合わせてください。重要な概念は数式で表現し、それ以外は文章で補足してください。"
    elif equation_frequency_level == 4:
        return "数式を積極的に活用し、概念や関係性を正確に表現してください。ただし、重要な説明は文章でも補足してください。"
    elif equation_frequency_level == 5:
        return "数式を最大限に活用してください。可能な限り多くの概念や関係性を数式で表現してください．"

### LLMによる章立ての出力

In [ ]:
book_node = book_graph.nodes[book_node_name]
next_parent_list = [book_node_name]

for depth in range(max_depth):
    parent_list = next_parent_list
    next_parent_list = []
    for parent_node_name in parent_list:
        for _, child_node_name in enumerate(book_graph.successors(parent_node_name)):
            parant_node = book_graph.nodes[parent_node_name]
            child_node = book_graph.nodes[child_node_name]


            if (child_node["needsSubdivision"] or child_node["n_pages"] >= max_output_pages) and depth < max_depth-1:

                # LLMによる出力
                prompt = PromptTemplate.from_template(prompt_section_list_creation)
                chain = prompt | llm

                result = chain.invoke(
                    {
                        "book_title": book_node["title"],
                        "book_summary": book_node["summary"],
                        "equation_frequency": get_equation_frequency(book_graph.graph["equation_frequency_level"]),
                        "target": child_node["title"],
                        "n_pages": child_node["n_pages"],
                        "section_summary": child_node["summary"]
                    }
                )

                # 出力を辞書型に変換
                section_json = extract_section_list(result.content)

                # グラフノードの作成・結果の格納
                book_graph.add_nodes_from([(child_node_name + "-" + str(idx+1), grandchild) for idx, grandchild in enumerate(section_json)])
                book_graph.add_edges_from([(child_node_name, child_node_name + "-" + str(idx+1)) for idx in range(len(section_json))])

                # 分節化した場合のみ次の親になる
                next_parent_list.append(child_node_name)

            elif not child_node["needsSubdivision"] or depth == max_depth-1:

                # グラフノードの作成・結果の格納
                book_graph.add_nodes_from([(child_node_name + "-p", {"content_file_path": child_node_name + "-p.tex"})])
                book_graph.add_edges_from([(child_node_name, child_node_name + "-p")])

            else:
                print("Error: needsSubdivision attribute is not set")

### 本文の内容の出力

#### 関数定義

In [ ]:
def extract_content_list(string_list):
    """
    この関数は、入力されたstring_listから特定のパターン（数字とハイフンの組み合わせで'-p'で終わる）
    にマッチする文字列のみを抽出し、新しいリストとして返す
    """
    pattern = r'(?:\d+-)*\d+-p'
    return [s for s in string_list if re.match(pattern, s)]

def custom_sort_key(s):
    """
    この関数は、文字列sを数字の部分で分割し、それらを整数のリストに変換する
    これにより、数値的な順序でソートするためのカスタムキーを生成する
    """
    parts = re.split(r'[-p]', s)
    return [int(part) for part in parts if part != '']

def sort_strings(string_list):
    """
    この関数は、入力されたstring_listを、custom_sort_key関数で定義された
    カスタムキーを使用してソートし、ソートされた新しいリストを返す
    """
    sorted_strings = sorted(string_list, key=custom_sort_key)
    return sorted_strings

def sort_leaf_nodes(book_graph):
    """
    この関数は、book_graphのリーフノードを特定のパターンに基づいて抽出し、
    数値的な順序で並び替えたリストを返します。

    引数:
    - book_graph: ノードに文字列ラベルを持つグラフオブジェクト

    戻り値:
    - sorted_content_str_list: 並び替えられた文字列のリスト
    """
    # book_graphのノードから、特定のパターンに一致する文字列を抽出する
    content_str_list = extract_content_list(list(book_graph.nodes))

    # 抽出された文字列リストをカスタムキーを使用して並び替える
    sorted_content_str_list = sort_strings(content_str_list)

    return sorted_content_str_list

def generate_outline(book_graph, book_node_name):
    """
    本文出力時に考慮させる章立てを出力する関数
    """

    toc_and_summary = "以下に章立てを示します．\n"

    # 本文の内容を持つノード（リーフノード）を順番に並び替え
    sorted_content_str_list = sort_leaf_nodes(book_graph)

    # 目次の作成
    toc_and_summary += "# " + book_graph.nodes[book_node_name]["title"] + "\n"
    toc_and_summary += book_graph.nodes[book_node_name]["summary"] + "\n"

    for heading_number_str in sorted_content_str_list:
        heading_number = custom_sort_key(heading_number_str)

        # 章
        if len(heading_number[1:]) == 0 or all(x == 1 for x in heading_number[1:]):
            node_name = "-".join(map(str, heading_number[0:1]))
            toc_and_summary += "## " + book_graph.nodes[node_name]["title"] + "\n"
            toc_and_summary += book_graph.nodes[node_name]["summary"] + "\n"

        # 節
        if (len(heading_number[2:]) == 0 and len(heading_number[:2]) > 1) or (len(heading_number[2:]) > 0 and all(x == 1 for x in heading_number[2:])):
            node_name = "-".join(map(str, heading_number[0:2]))
            toc_and_summary += "### " + book_graph.nodes[node_name]["title"] + "\n"
            toc_and_summary += book_graph.nodes[node_name]["summary"] + "\n"

        # 小節
        if (len(heading_number[3:]) == 0 and len(heading_number[:3]) > 2) or (len(heading_number[3:]) > 0 and all(x == 1 for x in heading_number[3:])):
            node_name = "-".join(map(str, heading_number[0:3]))
            toc_and_summary += "#### " + book_graph.nodes[node_name]["title"] + "\n"
            toc_and_summary += book_graph.nodes[node_name]["summary"] + "\n"

    return toc_and_summary

def slide_list_with_new_title_and_content(title, content, original_list):
    # スライドさせたリストを作成し、0番目に新しい要素（タイトルと内容の辞書）を追加
    slid_list = [{"title": title, "content": content}] + original_list[:-1]
    return slid_list

def generate_prompt_for_previous_sections(previous_sections_content_list, n_previous_sectinos):

    prompt = ""

    for i in range(n_previous_sectinos):
        i_previous_section = previous_sections_content_list[i]
        title = i_previous_section["title"]
        content = i_previous_section["content"]
        prompt += f"{i+1}つ前のパートのタイトルは{title}で、内容は以下です．\n{content}\n"

    return prompt

#### 本文出力

In [ ]:
# 本文出力時に考慮させる章立ての出力
toc_and_summary = ""
if do_consider_outline:
    toc_and_summary = generate_outline(book_graph, book_node_name)

# 過去の出力を保持するための文字列リストの初期化
previous_sections_content_list = [{"title": "", "content": ""} for i in range(n_previous_sections)]

# 本文の内容を持つノード（リーフノード）を順番に並び替え
sorted_content_str_list = sort_leaf_nodes(book_graph)

# 本文の追加
for heading_number_str in sorted_content_str_list:
    heading_number = custom_sort_key(heading_number_str)

    node_name = "-".join(map(str, heading_number))
    node = book_graph.nodes[node_name]

    # LLMによる出力
    prompt = PromptTemplate.from_template(prompt_content_creation)
    chain = prompt | llm

    result = chain.invoke(
        {
            "book_title": book_node["title"],
            "book_summary": book_node["summary"],
            "toc_and_summary": toc_and_summary,
            "previous_sections": generate_prompt_for_previous_sections(previous_sections_content_list, n_previous_sections),
            "equation_frequency": get_equation_frequency(book_graph.graph["equation_frequency_level"]),
            "target": node["title"],
            "n_pages": node["n_pages"],
            "section_summary": node["summary"]
        }
    )

    # 出力をファイルに保存
    contents_tex = extract_section_content(result.content)
    with open(node_name + "-p.tex", mode='w', encoding='UTF-8') as f:
        f.write(contents_tex)

    # 過去の出力をn_previous_sections分保持
    if do_consider_previous_sections:
        previous_sections_content_list = slide_list_with_new_title_and_content(node["title"], contents_tex, previous_sections_content_list)

## 本グラフの表示

In [ ]:
pos = graphviz_layout(book_graph, prog="dot")

# matplotlib settings
fig = plt.figure(figsize=(20, 10), dpi=300)
ax = fig.add_subplot(1, 1, 1)

# リーフノード（本文ノード）を赤く
node_color_list = ["r" if "p" in s else "w" for s in list(book_graph.nodes)]

# draw the network
nx.draw(book_graph,
        ax=ax,
        pos=pos,
        with_labels=True,
        node_size=300,
        node_color=node_color_list,
        alpha=0.3,
        node_shape='.',
        width=0.5)

print("bookから出ている矢印の先が章を表していて、その下が節、その下が小節です。\nこれらのノードは、その章・節のタイトルや概要を情報として持っています。\nそして、赤い丸で示されている末端のノードは、本文の内容を情報として持っているノードになります。")

## 本の作成

### latexmkrcファイルの作成

In [ ]:
# ホームディレクトリのパスを取得
home_dir = os.path.expanduser("~")

# .latexmkrcファイルのパスを作成
latexmkrc_path = os.path.join(home_dir, ".latexmkrc")

# latexmkrcファイルの中身
content = '''$latex = 'platex -synctex=1 -halt-on-error -interaction=nonstopmode -file-line-error %O %S';
$bibtex = 'pbibtex %O %S';
$biber = 'biber --bblencoding=utf8 -u -U --output_safechars %O %S';
$makeindex = 'mendex %O -o %D %S';
$dvipdf = 'dvipdfmx %O -o %D %S';

$max_repeat = 5;
$pdf_mode = 3;'''

# ファイルを作成して内容を書き込む
try:
    with open(latexmkrc_path, "w") as file:
        file.write(content)
    print(f".latexmkrc file has been created successfully at {latexmkrc_path}")
except IOError as e:
    print(f"An error occurred while creating the file: {e}")

### LaTeXで本文の作成

### LaTeXドキュメントの作成

In [ ]:
# pylatexにより、PDFを作成
geometry_options = {"tmargin": "3cm", "lmargin": "3cm"}
doc = Document(documentclass="jsreport", geometry_options=geometry_options)

# プリアンブル・タイトルの追加
doc.packages.append(Package('amsmath'))
doc.packages.append(Package('amssymb'))
doc.packages.append(Package('amsfonts'))
doc.packages.append(Package('mathtools'))
doc.packages.append(Package('bm'))
doc.packages.append(Package('physics'))
doc.packages.append(Package('inputenc', options="utf8"))
doc.packages.append(Package('listings'))
doc.packages.append(Package('jvlisting'))
doc.packages.append(Package('color'))
doc.packages.append(Package('underscore', options="strings"))
doc.preamble.append(Command("title", book_graph.nodes[book_node_name]["title"]))
doc.preamble.append(Command("date", NoEscape(r"\today")))
doc.append(NoEscape(r"\maketitle"))
doc.append(NoEscape(r"\tableofcontents"))
doc.append(NoEscape(r'\lstset{ backgroundcolor={\color[gray]{.90}}, breaklines = true, breakindent = 10pt, basicstyle = \ttfamily\scriptsize, commentstyle = {\itshape \color[cmyk]{1,0.4,1,0}}, classoffset = 0, keywordstyle = {\bfseries \color[cmyk]{0,1,0,0}}, stringstyle = {\ttfamily \color[rgb]{0,0,1}}, frame = TBrl, framesep = 5pt, numbers = left, stepnumber = 1, numberstyle = \tiny, tabsize = 4, captionpos = t}'))

In [ ]:
# 本文の内容を持つノードを順番に並び替え
content_str_list = extract_content_list(list(book_graph.nodes))
sorted_content_str_list = sort_strings(content_str_list)

# 本文の追加
for heading_number_str in sorted_content_str_list:
    heading_number = custom_sort_key(heading_number_str)

    # 章の見出しの追加
    if len(heading_number[1:]) == 0 or all(x == 1 for x in heading_number[1:]):
        node_name = "-".join(map(str, heading_number[0:1]))
        with doc.create(Chapter(book_graph.nodes[node_name]["title"], label=False)):
            doc.append(NoEscape(book_graph.nodes[node_name]["summary"].replace("\\\\","\\")))

    # 節の見出しの追加
    if (len(heading_number[2:]) == 0 and len(heading_number[:2]) > 1) or (len(heading_number[2:]) > 0 and all(x == 1 for x in heading_number[2:])):
        node_name = "-".join(map(str, heading_number[0:2]))
        with doc.create(Section(book_graph.nodes[node_name]["title"], label=False)):
            doc.append(NoEscape(book_graph.nodes[node_name]["summary"].replace("\\\\","\\")))

    # 小節の見出しの追加
    if (len(heading_number[3:]) == 0 and len(heading_number[:3]) > 2) or (len(heading_number[3:]) > 0 and all(x == 1 for x in heading_number[3:])):
        node_name = "-".join(map(str, heading_number[0:3]))
        with doc.create(Subsection(book_graph.nodes[node_name]["title"], label=False)):
            doc.append(NoEscape(book_graph.nodes[node_name]["summary"].replace("\\\\","\\")))

    # 本文の追加
    tex_file_path = book_graph.nodes[heading_number_str]["content_file_path"]
    with open(tex_file_path, "r", encoding='UTF-8') as file:
        tex_content = file.read()
        doc.append(NoEscape(tex_content))

### PDFファイルの作成

In [ ]:
# コンパイル．/content/textbook.pdfというファイルが作成される
n_displayed_line = 10
isErrorOccured = False

try:
    doc.generate_pdf(book_node["title"], compiler="latexmk", clean_tex=False)
except:
    log_file_path = "/content/" + book_node["title"] + ".log"

    # ログファイルの内容を読み込む
    with open(log_file_path, "r", encoding="utf-8",errors="ignore") as file:
        lines = file.readlines()

    # エラー出力パターンを定義する (file-line-error オプション形式)
    pattern = r'([a-zA-Z0-9_.]+):(\d+): (.+)'

    for i, line in enumerate(lines):
        match = re.search(pattern, line)
        if match:
            isErrorOccured = True
            print(f"Line: {match.group(2)}")
            print(f"Message: \n  {match.group(3)}")
            for j in range(i+1, min(i+n_displayed_line, len(lines))):
                print(lines[j].strip())
            print("-" * 40)

if isErrorOccured:
    files.download("/content/" + book_node["title"] + ".tex")
    raise Exception("TeXファイルをコンパイル時にエラーが発生しました．再度実行するか，上記のエラー内容を参照して，エラー修正後にコンパイルしてください．")

In [ ]:
# ダウンロード
if tex_output:
    files.download("/content/" + book_node["title"] + ".tex")

if pdf_output:
    files.download("/content/" + book_node["title"] + ".pdf")

### Markdownファイルの作成

In [ ]:
def clean_markdown_content(content):
    # はじめの見出しより前の部分（LaTeXの設定に関する部分）を削除
    content = content.split('#', 1)[1]

    #  改行またはスペースが前後にある場合のみ、%を削除
    content = re.sub(r"(?<=\s)%|%(?=\s)", "", content)

    # %が末尾に現れた場合のみ削除する。
    content = re.sub(r"%\s*$", "", content)

    # 複数の改行を１つの改行に置き換える
    content = re.sub(r"\n{2,}", "\n\n", content)

    return content

def convert_to_latex_to_katex(content):
    # align, align* 環境の変換
    content = re.sub(r'\\begin{align\*?}', r'$$\n\\begin{aligned}', content)
    content = re.sub(r'\\end{align\*?}', r'\\end{aligned}\n$$', content)

    # equation, equation* 環境の変換
    content = re.sub(r'\\begin{equation\*?}', r'$$', content)
    content = re.sub(r'\\end{equation\*?}', r'$$', content)

    return content

def transform_code_blocks(content):

    # languageオプションを抽出して対応
    def replace_lstlisting(match):
        options = match.group(1)
        code = match.group(2)
        # languageオプションを探す
        lang_match = re.search(r'language=([a-zA-Z]+)', options)
        lang = lang_match.group(1) if lang_match else ''
        # Markdown形式に変換
        return f'```{lang}\n{code}\n```'

    # lstlistingの変換処理
    content = re.sub(r'\\begin{lstlisting}\[(.*?)\](.*?)\\end{lstlisting}', replace_lstlisting, content, flags=re.DOTALL)

    return content

In [ ]:
if md_output:
    %pip install latex2markdown
    import latex2markdown

    with open("/content/" + book_node["title"] + ".tex", "r") as f:
        latex_string = f.read()

    # LaTeXをMarkdownに変換
    processed_latex_str = transform_code_blocks(latex_string)
    l2m = latex2markdown.LaTeX2Markdown(processed_latex_str)
    markdown_string = l2m.to_markdown()

    # Markdownのテキストを整形
    cleaned_md_str = clean_markdown_content(markdown_string)
    katex_md_str = convert_to_latex_to_katex(cleaned_md_str)

    with open("/content/" + book_node["title"] + ".md", "w") as f:
        f.write(katex_md_str)
    files.download("/content/" + book_node["title"] + ".md")